In [1]:
import numpy as np
import time

In [2]:
def check_data(X,y):
  if len(X) != len(y):
    return False, "Length of the answer vector doesn't fit the number of data points.\n"
  for i in range(1,len(X)):
    if len(X[i]) != len(X[0]):
      return False, "The row at the index "+str(i)+" seems to be missing an observation.\n"
  for i in range(len(y)):
    if y[i] < 0 or y[i] > 1:
      return False, "The answer at the index "+str(i)+" doesn't indicate a binary class nor does it indicate a probability of belonging to one.\n"
  return True, ""

In [4]:
def calc_loglike(X, Y, beta):
  """
  Calculates the value of the log-likelihood for a given training data,
  answer vector and model parameters
  :param X: a 2d numpy array containing the experiment matrix.
  :param Y: a 1d numpy array containing the answer vector.
  :param beta: a 1d numpy array containing the parameters of the model.
  :return: a single float with the value of the log-likelihood function.
  """
  Xbeta = X @ beta
  return Y @ Xbeta - np.sum(np.log(1+np.exp(Xbeta)))

In [5]:
def fit_with_IWLS(data, answers, intercept: bool = True,
                  relevant_variables = None, additional_interactions = None,
                  l2_reg: float = 1.0, beta0_gen = None,
                  max_iterations: int = 500, min_step_norm: float = 1e-4,
                  max_time: float = 3600.0, check_data: bool = False):
  """
  Calculates the coefficients of a Logistic Regression model using Iterative
  Weighted Least Squares method.
  :param data: The data on which the model will be fit.
  :param answers: The vector with answers (numbers belonging to the
    [0,1] interval).
  :param intercept: If True, the model will fit an intercept (meaning beta' @ x
    shall be replaced with beta' @ x + beta0 in all calculations).
  :param relevant_variables: A collection of indices, indicating on which
    columns of the data should the model be built.
    If None, all columns will be used.
  :param additional_interactions: A collection of pairs of indices, indicating
    which column element-wise products should be using for building the model.
    If None, no such variables will be considered.
  :param l2_reg: The strength of ridge regularization (the coefficient of
    the ridge penalty). 0 means no regularization. 1 is the default, same as
    in the scikit-learn implementation.
  :param beta0_gen: A generator used to determine the starting values of
    coefficients. Should include .generate(n: int) method, returning a numpy
    array of length n filled with floats.
    If None, all coefficients will be initialized to zeros.
  :param max_iterations: The maximum number of iterations the algorhithm will
    perform before stopping and proposing a solution.
    By default, 100 in accordance to scikit-learn implementation.
  :param min_step_norm: The minimum value for the euclidian norm of the change
    of a parameter vector in a single step. If the difference between
    iterations falls below that number, the algorhithm will stop and propose
    a solution.
  :param max_time: The maximum time the procedure can run in seconds. Once
    exceeded, the iterating will stop and the solution will be proposed.
  :param check_data: If True, the format of data and answers will be examined
    prior to running the algorhithm.

  :return: A numpy array containing the proposed coefficients and a dictionary,
    labeling said coefficients, a list containing the coefficients after each
    iteration and a list containing the values of the log-likelihood function
    after each iteration.
  """
  # Ensuring the correct dimensionality of the data.
  if check_data:
    status, message = check_data(data, answers)
    assert status, message
  assert len(data) == len(answers), "For every data point, there has to be a correct class specified.\n"
  n = len(data)

  # Filling up the default values of parameters.
  if additional_interactions is None:
    additional_interactions = []
  if relevant_variables is None:
    relevant_variables = np.arange(len(data[0]))

  ### Constructing the experiment matrix and labels for it.
  Y = np.array(answers)
  X = []
  labels = []
  for index in relevant_variables:
    X.append(np.array(data[:,index]).astype(float))
    labels.append("X"+str(index))

  for index1, index2 in additional_interactions:
    X.append(np.array(data[:,index1]).astype(float) * np.array(data[:,index2]).astype(float))
    labels.append("X"+str(index1)+"X"+str(index2))

  if intercept:
    X.append(np.ones(n))
    labels.append("intercept")

  X = np.column_stack(X)
  p = len(labels)
  # If the penalty is the l2_reg times the sum of squares of coefficients, this
  # is the matrix of the second order derivatives with respect to the coefs.
  penalty_hessian = 2 * l2_reg * np.eye(p)

  ### Initializing coefficients.
  if beta0_gen is None:
    beta = np.zeros(p)
  else:
    beta = beta0_gen.generate(p)

  start = time.time()
  beta_hist = []
  loglike_hist = []
  beta_hist.append(beta)
  loglike_hist.append(calc_loglike(X, Y, beta))
  ### Iterating the main algorhithm.
  for _ in range(max_iterations):
    P = X @ beta
    P = np.exp(P) / (np.exp(P) + 1)
    W = np.diag(P * (1 - P))

    # deriv is the derivative of the minus log-like + penalty with respect to beta
    deriv =  X.transpose() @ (P-Y) + 2 * l2_reg * beta
    # hessian is the matrix of second order derivatives of the previously mentioned function
    hessian = X.transpose() @ W @ X + penalty_hessian

    # In order to avoid numerical complexity of the matrix inversion,
    # new beta is defined as a solution to a linear equation.
    beta_new = np.linalg.solve(hessian, hessian @ beta - deriv)

    diff = beta - beta_new
    diff_norm = np.sqrt(diff @ diff)
    beta = beta_new

    beta_hist.append(beta)
    loglike_hist.append(calc_loglike(X, Y, beta))

    if diff_norm < min_step_norm:
      break
    curr = time.time()
    if curr - start > max_time:
      break

  ### Creating the coefficient dictionary.
  beta_dict = {}
  for i in range(p):
    beta_dict[labels[i]] = beta[i]

  return beta, beta_dict, beta_hist, loglike_hist

**Example**

In [6]:
def generate_fake_data(n: int):
  X = np.random.normal(loc=0,scale=1,size=(n,3))
  Y = X[:,0] + (X[:,1] * X[:,2])
  Y = (Y > 1).astype(int)
  return X, Y

In [7]:
X, Y = generate_fake_data(10)
print(X)
print(Y)

[[ 0.12777544 -0.94382539  0.34109503]
 [-1.97898473  0.3166541   2.13205413]
 [ 0.04992092  0.18904967 -0.06868558]
 [-0.3573216  -0.42572623 -1.1094144 ]
 [ 2.04911748 -0.25506455  0.50892927]
 [ 2.21879392  0.20278518  1.24085252]
 [ 1.19691487  1.00489141  0.69139772]
 [-0.20357283 -0.27923804 -0.26133622]
 [ 0.46453406 -1.22609239  0.73921278]
 [-0.81565596 -1.2438685   0.56181934]]
[0 0 0 0 1 1 1 0 0 0]


In [8]:
X, Y = generate_fake_data(1000)
interactions = [(0,1), (1,2), (2,0)]

In [9]:
beta1, beta1_dict, _, _ = fit_with_IWLS(X, Y, intercept=False, max_iterations=500)
beta2, beta2_dict, _, _ = fit_with_IWLS(X, Y, intercept=True, additional_interactions=interactions, max_iterations=500)

In [10]:
print(beta1_dict)
print(beta2_dict)

{'X0': 1.2858350777070253, 'X1': -0.17847408348876884, 'X2': 0.05173706992080961}
{'X0': 4.010084935918466, 'X1': -0.0914026690534419, 'X2': 0.056193730148987486, 'X0X1': 0.016660571525063544, 'X1X2': 3.6006955731571835, 'X2X0': -0.08156258812843332, 'intercept': -3.8712434655312964}
